In [105]:
import pandas as pd
import datetime

In [50]:
eventTypes = ['accidentsAndIncidents', 'roadwork', 'precipitation', 'deviceStatus', 'obstruction', 'trafficConditions']

In [2]:
trials = pd.read_csv("prediction_trials.tsv", sep="\t")

In [85]:
trials.head()

,trial_id,nw_lat,nw_lon,se_lat,se_lon,start,end
0,p_00000001,39.182985,-76.798663,39.170238,-76.778150,2015-02-01T00:00:00-05:00,2015-03-01T00:00:00-05:00
1,p_00000002,39.303607,-76.829335,39.296709,-76.806935,2016-01-08T00:00:00-05:00,2016-01-22T00:00:00-05:00
2,p_00000003,38.946840,-76.783484,38.931581,-76.761971,2015-03-01T00:00:00-05:00,2015-04-01T00:00:00-05:00
3,p_00000004,38.996686,-77.159555,38.982995,-77.140682,2015-01-01T00:00:00-05:00,2015-02-01T00:00:00-05:00
4,p_00000005,39.283366,-76.562381,39.265534,-76.544242,2016-04-05T00:00:00-05:00,2016-04-14T00:00:00-05:00


In [80]:
def findYear(row):
    years = ['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
    for year in years:
        if(year in str(row['closed_tstamp'])):
            return int(year)
    for year in years:
        if(year in str(row['created_tstamp'])):
            return int(year)
    for year in years:
        if(year in str(row['confirmed_tstamp'])):
            return int(year)

In [54]:
events = pd.read_csv("events_train.tsv", sep="\t")

In [59]:
events = events[events.event_type.isin(eventTypes)]

In [61]:
events.groupby('event_type').size()

event_type
accidentsAndIncidents    550079
deviceStatus              38997
obstruction               32971
precipitation              4330
roadwork                 155896
trafficConditions         55663
dtype: int64

In [62]:
events['year'] = events.apply(lambda row : findYear(row), axis = 1)

In [99]:
print(trials.shape)

(328416, 7)


In [183]:
yearsTrain = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]  
for ind in range(trials.shape[0]):
    current_trial = trials.loc[ind,'trial_id']
    nw_lat = trials.loc[ind, 'nw_lat']
    nw_lon = trials.loc[ind, 'nw_lon']
    se_lat = trials.loc[ind, 'se_lat']
    se_lon = trials.loc[ind, 'se_lon']
    events_f = events[(events.latitude <= nw_lat) & (events.latitude >= se_lat) & (events.longitude >= nw_lon) & (events.longitude <= se_lon)]
    
    efg = events_f.groupby(['year', 'event_type']).size().unstack()
    efg.fillna(0, inplace=True)
    for event in eventTypes:
        if(not(event in efg.columns)):
            efg[event] = 0
        
    for year in yearsTrain:
        if(not(year in efg.index)):
            efg.loc[year] = 0
        
    efg.sort_index(inplace = True)
    efg = efg.reindex_axis(sorted(efg.columns), axis=1)
    
    efg.to_csv('event_counts/' + current_trial + '.csv', sep=',' , header = False)
    
    if(ind%100 == 0):
        print('completed row ' + str(ind))

completed row 0
completed row 100
completed row 200
completed row 300
completed row 400
completed row 500
completed row 600
completed row 700
completed row 800
completed row 900
completed row 1000
completed row 1100
completed row 1200
completed row 1300
completed row 1400
completed row 1500
completed row 1600
completed row 1700
completed row 1800
completed row 1900
completed row 2000
completed row 2100
completed row 2200
completed row 2300
completed row 2400
completed row 2500
completed row 2600
completed row 2700
completed row 2800
completed row 2900
completed row 3000
completed row 3100
completed row 3200
completed row 3300
completed row 3400
completed row 3500
completed row 3600
completed row 3700
completed row 3800
completed row 3900
completed row 4000
completed row 4100
completed row 4200
completed row 4300
completed row 4400
completed row 4500
completed row 4600
completed row 4700
completed row 4800
completed row 4900
completed row 5000
completed row 5100
completed row 5200
compl